# Absorption logging in Union components
It is possible to record absorption that occurs within a Union system by using absorption loggers. There are several different absorption loggers that behave in similar ways. Here we will use absorption loggers for two purposes:
- Visualize absorption in the system to better understand it
- Making a neutron detector to generate realistic data

First we set up the quiz and instrument.

In [ ]:
import mcstasscript as ms
import load_quiz

In [ ]:
instrument = ms.McStas_instr("exercise_4", input_path="run_folder")
instrument.settings(output_path="data_folder/exercise_4")

instrument.add_component("init", "Union_init") # Necessary for current version of Union in McStas 3.X

quiz = load_quiz.exercise_4()

## Exercise instrument

We will be working with a simple SANS instrument which consists of:
- Virtual source that sends neutrons forward with very small angular spread
- SANS sample (can be enabled and disabled)
- Beamstop (can be enabled and disabled)
- Detector position with its distance to sample controlled by parameter
- Union system with He3 gas in aluminium tube at detector position with y>0

No need to read all the details of the instrument before starting the exercises.

In [ ]:
src = instrument.add_component("source", "Source_div")
src.xwidth = 0.01
src.yheight = 0.02
src.focus_aw = 0.0001
src.focus_ah = 0.0001
src.flux = 1E13
instrument.add_parameter("double", "wavelength", value=6.0, comment="[AA]  Mean wavelength of neutrons")
instrument.add_parameter("double", "d_wavelength", value=3.0, comment="[AA]  Wavelength spread of neutrons")
src.lambda0 = "wavelength"
src.dlambda = "d_wavelength"

sample_position = instrument.add_component("sample_position", "Arm")
sample_position.set_AT([0, 0, 1], RELATIVE=src)

enable_sample = instrument.add_parameter("double", "enable_sample", value=0,
                                         comment="0 for nothing, 1 for SANS sample")
enable_sample.add_option(0, options_are_legal=True)
enable_sample.add_option(1, options_are_legal=True)

sample_conventional = instrument.add_component("sample_conventional", "SANS_spheres2")
sample_conventional.xwidth = 0.02
sample_conventional.yheight = 0.02
sample_conventional.zthick = 0.001
sample_conventional.sc_aim = 0.95
sample_conventional.sans_aim = 0.95
R_par = instrument.add_parameter("double", "SANS_R", value=150.0,
                                 comment="[AA]  Radius of scattering hard spheres")
sample_conventional.R = R_par
sample_conventional.set_WHEN("enable_sample == 1")
sample_conventional.set_AT(0, RELATIVE="sample_position")

dist = instrument.add_parameter("double", "detector_distance", value=3.0,
                                comment="[m] Sample_detector_distance")
detector_position = instrument.add_component("detector_position", "Arm")
detector_position.set_AT([0, 0, dist], RELATIVE=sample_position)

enable_beamstop = instrument.add_parameter("double", "enable_beamstop", value=0,
                                           comment="0 for nothing, 1 for beamstop")
enable_beamstop.add_option(0, options_are_legal=True)
enable_beamstop.add_option(1, options_are_legal=True)

beamstop = instrument.add_component("beamstop", "Beamstop")
beamstop.set_AT(-0.2, detector_position)
beamstop.set_WHEN("enable_beamstop == 1")
beamstop.set_parameters(xwidth=0.021, yheight=0.021)

# Set up Al material with incoherent and powder
Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.sigma = "4*0.0082"
Al_incoherent.packing_factor = 1
Al_incoherent.unit_cell_volume = 66.4

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = "\"Al.laz\""

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = "100*4*0.231/66.4"

# Set up He3 material with incoherent scattering
def mu_gas(sigma, bars, temperature_C):
    pressure_Pa = bars*1E5
    number_density_mol_per_m3 = pressure_Pa/(8.3145*(temperature_C + 273.15))
    number_density_per_m3 = number_density_mol_per_m3*6.022E23
    number_density_per_A3 = number_density_per_m3/1E30
    return sigma*number_density_per_A3*100

He3_pressure_bars = 3
He3_temperature_C = 25

He3_inc = instrument.add_component("He3_inc", "Incoherent_process")
He3_inc.sigma = mu_gas(1.6, bars=He3_pressure_bars, temperature_C=He3_temperature_C)
He3_inc.unit_cell_volume = 100

He3 = instrument.add_component("He3", "Union_make_material")
He3.process_string = '"He3_inc"'
He3.my_absorption = mu_gas(5333, bars=He3_pressure_bars, temperature_C=He3_temperature_C)

# Create detector casing with gas volume inside
casing = instrument.add_component("Al_container", "Union_cylinder")
casing.set_parameters(yheight=0.5, radius=0.03, p_interact=0.2,
                      material_string='"Al"', priority=300)
casing.set_AT([0, 0.5*casing.yheight, 0], detector_position)

He3_gas = instrument.add_component("He3_gas", "Union_cylinder")
He3_gas.set_AT_RELATIVE(casing)
He3_gas.set_parameters(yheight=casing.yheight-0.02, radius=casing.radius-4E-3,
                            material_string='"He3"', priority=310)

# Reference detectors
ref = instrument.add_component("reference", "PSDlin_monitor", after=beamstop)
ref.set_parameters(xwidth=2*He3_gas.radius, yheight=He3_gas.yheight, vertical=1, 
                   nbins=300, filename='"reference.dat"', restore_neutron=1)
ref.set_AT([0, 0.5*casing.yheight, 0], RELATIVE=detector_position)

# Parmeters for time of flight limits
instrument.add_declare_var("double", "t_min")
instrument.add_declare_var("double", "t_max")

instrument.append_initialize("t_min = (wavelength - d_wavelength)*(1.0 - 0.05 + detector_distance)/(K2V*2*PI);")
instrument.append_initialize("t_max = (wavelength + d_wavelength)*(1.0 + 0.2 + detector_distance)/(K2V*2*PI);")
instrument.add_declare_var("char", "options", array=256)
instrument.append_initialize('sprintf(options,"square, y bins=200, t limits=[%g %g] bins=300",t_min,t_max);')

ref = instrument.add_component("reference_tof", "Monitor_nD", after=beamstop)
ref.set_parameters(xwidth=2*He3_gas.radius, yheight=He3_gas.yheight,
                   filename='"reference_tof.dat"', restore_neutron=1)
ref.options = "options"
ref.set_AT([0, 0.5*casing.yheight, 0], RELATIVE=detector_position)


In [ ]:
instrument.show_diagram()

### Question 1
Add a Union_abs_logger_1D_space component called signal placed relative to the He3_gas component with the same yheight. Set the following parameters:
- n: 300 (number of bins)
- target_geometry: He3_gas
- reasonable filename

In [ ]:
# Your code here

In [ ]:
quiz.question_1(instrument)

### Question 2
Add a similar logger with time of flight information, this component is called Union_abs_logger_1D_space_tof and should be placed in the same way. Call this component "signal_tof". The time of flight limits have been calculated and are stored in the McStas variables t_min and t_max which should be assigned to the proper component parameters.

- n: 200 (number of spatial bins)
- time_bins: 300 (number of time bins)
- time_min: t_min parameter
- time_max: t_max parameter
- Reasonable filename (different from the first)

In [ ]:
# Your code here

In [ ]:
quiz.question_2(instrument)

### Question 3
Add a Union_abs_logger_2D_space logger to visualize the absorption location within the system. Place it relative to the He3_gas component and use the following parameters:
- D_direction_1: '"z"'
- D1_min: -4 cm
- D1_max: 4 cm
- n1: 300
- D_direction_2: '"y"'
- D2_min: -26 cm
- D2_max: 26 cm
- n2: 300
- Reasonable filename (different from the other absorption loggers)

In [ ]:
# Your code here

In [ ]:

quiz.question_3(instrument)

## Finishing up the simulation
We now add the master and stop component so the Union system is ready for simulation.

In [ ]:
master = instrument.add_component("master", "Union_master")
stop = instrument.add_component("stop", "Union_stop")

# Running the simulation
In this case we use a slightly different way to set up the simulation interface that allow us to extract the generated data. In the cell below the interface you can compare the Union detector with the reference, it will grab the last data set simulated with the widget.

In [ ]:
import mcstasscript.jb_interface as ms_widget
%matplotlib widget
sim_interface = ms_widget.SimInterface(instrument)
sim_interface.show_interface()

In [ ]:
data = sim_interface.get_data()

ref = ms.name_search("reference", data)
union = ms.name_search("signal", data)

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, gridspec_kw={'height_ratios': [2.5, 1]}, figsize=(8, 6))
ax1.errorbar(ref.xaxis, ref.Intensity, yerr=ref.Error, label="reference")
ax1.errorbar(union.xaxis, union.Intensity, yerr=union.Error, label="Union")
ax1.set_ylabel(union.metadata.ylabel)
ax1.set_yscale("log")
ax1.legend()

ax2.plot(ref.xaxis, [1]*len(ref.xaxis), '--k') # reference line
ax2.errorbar(ref.xaxis, union.Intensity/ref.Intensity)
ax2.set_yscale("log")
ax2.set_ylabel("Ratio Union/Ref")
ax2.set_xlabel(union.metadata.xlabel)
fig.show()

# Compare 2D plots side by side for tof monitors
ref_tof = ms.name_search("reference_tof", data)
ref_tof.set_title("Ref")
union_tof = ms.name_search("signal_tof", data)
union_tof.set_title("Union")
ms.make_sub_plot([ref_tof, union_tof], figsize=(8, 4), log=True, orders_of_mag=10)

### Simulation note
These next questions concern results from the simulation, you may need to run with 1E7 to 1E8 rays to see the discussed effects.

### Question 4
Run a simulation without the SANS sample and without the beamstop. On the detectors, what is the y value of the direct beam?

In [ ]:
quiz.question_4() # Insert value

### Question 5
Run the simulation with the SANS sample and without a beamstop. Where is the largest discrepancy between the reference and the Union signal?

- A: The bottom of the detector (low y)
- B: The center of the detector (0 y)
- C: The top of the detector (high y)

In [ ]:
quiz.question_5()

### Question 6
Why do we see this discrepancy?

- A: The Union detector is more efficient than the reference
- B: The direct beam hits the detector and scatters into it
- C: Scattering of the SANS signal within the detector

In [ ]:
quiz.question_6()

### Question 7
Run the simulation with both the SANS sample and beamstop enabled at high wavelengths (lowest wavelength at 3 or higher). How would you describe the relationship between the reference and Union signal?

- A: They agree
- B: There is a scaling difference
- C: There are large unexpected difference

In [ ]:
quiz.question_7()

### Question 8
Run the simulation with both the SANS sample and beamstop enabled at high wavelengths (highest wavelength at 2 or lower). How is the agreement between the reference and Union signal?

- A: They agree
- B: There is a scaling difference
- C: There are large unexpected difference

In [ ]:
quiz.question_8()

### Question 9
Run the simulation with a very narrow wavelength spread around 6 AA and compare the two tof diagrams.
- A: The reference line is broader
- B: The union line is broader
- C: They are similar
- D: The reference data has a lot of background
- E: The Union data has a lot of background
- F: The Union data has an unexpected feature

In [ ]:
quiz.question_9()

### Question 10
Run the simulation with a very narrow wavelength spread around 1 AA and compare the two tof diagrams. Increase the detector distance to 10 m to see some more interesting part of the SANS signal at this low wavelength.

- A: The reference line is broader
- B: The union line is broader
- C: They are similar
- D: The reference data has a lot of background
- E: The Union data has a lot of background
- F: The Union data has an unexpected feature

In [ ]:
quiz.question_10()